# Fine-tuning transformer

### Getting the data

#### Train/test data

In [ ]:
!mkdir data
!mkdir data/history
!wget --no-check-certificate 'https://drive.usercontent.google.com/download?id=1iqg1FIPfbrZWlung6gZqve1MeQWc0Je4&export=download&authuser=1&confirm=t' -O './data/dataset.csv'

--2024-12-18 16:19:51--  https://drive.usercontent.google.com/download?id=1iqg1FIPfbrZWlung6gZqve1MeQWc0Je4&export=download&authuser=1&confirm=t
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.197.132, 2607:f8b0:400e:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.197.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 218678075 (209M) [application/octet-stream]
Saving to: './data/dataset.csv'

./data/dataset.csv  100%[===================>] 208.55M   111MB/s    in 1.9s    

2024-12-18 16:19:55 (111 MB/s) - './data/dataset.csv' saved [218678075/218678075]



In [ ]:
import pandas as pd


df = pd.read_csv('./data/dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22224 entries, 0 to 22223
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   title                               22224 non-null  object 
 1   location                            22224 non-null  object 
 2   company                             22224 non-null  object 
 3   skills                              14384 non-null  object 
 4   description                         22224 non-null  object 
 5   salary_from                         22224 non-null  float64
 6   source                              22224 non-null  object 
 7   experience_from                     22224 non-null  float64
 8   description_no_numbers              22224 non-null  object 
 9   log_salary_from                     22224 non-null  float64
 10  description_no_numbers_with_skills  22224 non-null  object 
 11  experience_to_adjusted_10           22224

### Extra dependencies

In [ ]:
%pip install -U sentence-transformers datasets -qqq

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nltk


nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

### Service functions

In [ ]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import gc
import re
import os
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, LoggingHandler
from sentence_transformers import models, util, datasets, evaluation, losses
from numba import cuda


os.environ["WANDB_DISABLED"] = "true"


def memory_cleanup():
    "Clean up memory"
    gc.collect()
    torch.cuda.empty_cache()


def get_sentence_lengths(text):
    "Get number of words in each sentence in the text"
    pattern = r'(?<=[.!?])'
    sentences = re.split(pattern, text)

    # remove empty strings
    sentences = [sentence for sentence in sentences if len(sentence) > 0]

    # get number of words in each sentence
    sentence_lengths = [len(sentence.split()) for sentence in sentences]

    return sentences, sentence_lengths


def set_seed(seed: int) -> None:
    "Set seed for reproducibility"
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def train_tsdae_bert(model_name, train_sentences):
    """Train a denoising auto-encoder model with BERT model.
    more examples at https://sbert.net/examples/unsupervised_learning/TSDAE/README.html"""
    word_embedding_model = models.Transformer(model_name)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), "cls")
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    # Create the special denoising dataset that adds noise on-the-fly
    train_dataset = datasets.DenoisingAutoEncoderDataset(train_sentences)

    # DataLoader to batch your data
    train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

    # Use the denoising auto-encoder loss
    train_loss = losses.DenoisingAutoEncoderLoss(
        model, decoder_name_or_path=model_name, tie_encoder_decoder=True,
    )

    # Call the fit method
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        weight_decay=0,
        scheduler="constantlr",
        optimizer_params={"lr": 3e-5},
        show_progress_bar=True,
    )

    return model

Display model output

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t


def display_metrics_with_ci(history: dict):
    # plot mean and ci for train and test r2 for all seeds and all iterations, averaged over seeds
    seeds = list(history.keys())
    def mean_confidence_interval(data, confidence=0.95):
        n = len(data)
        m, se = np.mean(data), np.std(data) / np.sqrt(n)
        h = se * t.ppf((1 + confidence) / 2, n-1)
        return m, m-h, m+h

    r2_train_values = [history[seed]['train_r2'] for seed in seeds]
    r2_test_values = [history[seed]['test_r2'] for seed in seeds]

    r2_train_values = np.array(r2_train_values)
    r2_test_values = np.array(r2_test_values)

    r2_train_mean = np.mean(r2_train_values, axis=0)
    r2_test_mean = np.mean(r2_test_values, axis=0)

    r2_train_ci = np.array([mean_confidence_interval(r2_train_values[:, i]) for i in range(r2_train_values.shape[1])])
    r2_test_ci = np.array([mean_confidence_interval(r2_test_values[:, i]) for i in range(r2_test_values.shape[1])])

    plt.figure(figsize=(10, 6))
    plt.plot(r2_train_mean[1:], label='train')
    plt.fill_between(range(len(r2_train_mean[1:])), r2_train_ci[1:, 1], r2_train_ci[1:, 2], alpha=0.3)

    plt.plot(r2_test_mean[1:], label='test')
    plt.fill_between(range(len(r2_test_mean[1:])), r2_test_ci[1:, 1], r2_test_ci[1:, 2], alpha=0.3)
    plt.title('Mean R2 by epoch, with 95% CI')
    plt.xlabel('Epoch')
    plt.ylabel('R2')
    plt.legend()
    plt.show()

    mae_test_values = [history[seed]['test_mae'] for seed in seeds]
    rmse_test_values = [history[seed]['test_rmse'] for seed in seeds]

    mae_test_values = np.array(mae_test_values)
    rmse_test_values = np.array(rmse_test_values)

    mae_test_mean = np.mean(mae_test_values, axis=0)
    rmse_test_mean = np.mean(rmse_test_values, axis=0)

    mae_test_ci = np.array([mean_confidence_interval(mae_test_values[:, i]) for i in range(mae_test_values.shape[1])])
    rmse_test_ci = np.array([mean_confidence_interval(rmse_test_values[:, i]) for i in range(rmse_test_values.shape[1])])

    # get an index of the epoch, where the test R2 is the highest
    # get mean and CI for this epoch
    best_epoch = np.argmax(r2_test_mean)
    best_epoch_r2 = r2_test_mean[best_epoch]
    best_epoch_mae = mae_test_mean[best_epoch]
    best_epoch_rmse = rmse_test_mean[best_epoch]
    best_epoch_r2_ci = r2_test_ci[best_epoch]
    best_epoch_mae_ci = mae_test_ci[best_epoch]
    best_epoch_rmse_ci = rmse_test_ci[best_epoch]

    print(f'TEST METRICS FOR THE BEST EPOCH (#{best_epoch+1})')
    print(f'R2: mean = {best_epoch_r2:.4f}, 95% CI = [{best_epoch_r2_ci[1]:.4f}, {best_epoch_r2_ci[2]:.4f}]')
    print(f'MAE: mean = {best_epoch_mae:.4f}, 95% CI = [{best_epoch_mae_ci[1]:.4f}, {best_epoch_mae_ci[2]:.4f}]')
    print(f'RMSE: mean = {best_epoch_rmse:.4f}, 95% CI = [{best_epoch_rmse_ci[1]:.4f}, {best_epoch_rmse_ci[2]:.4f}]')

### Traning-related classes

#### Dataset

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

# Dataset for dual textual features
class DualTextDataset(Dataset):
    def __init__(self, df, text_col_1, text_col_2, targets, tokenizer, max_len):
        print('Creating the dataset...')
        # Pre-tokenize and store inputs
        self.tokenized_texts1 = tokenizer(df[text_col_1].tolist(), max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
        self.tokenized_texts2 = tokenizer(df[text_col_2].tolist(), max_length=max_len, padding="max_length", truncation=True, return_tensors="pt")
        self.targets = targets.tolist()
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        # Return only the slice for idx
        inputs1 = {key: val[idx] for key, val in self.tokenized_texts1.items()}
        inputs2 = {key: val[idx] for key, val in self.tokenized_texts2.items()}
        target = torch.tensor(self.targets[idx], dtype=torch.float)
        return inputs1, inputs2, target


#### Model

##### Double-head BERT with TSDAE pre-tuning and MLP

In [ ]:
import torch
import torch.nn as nn


class TSDAEDualBERTWithMLP(nn.Module):
    """two berts for two text features with MLP head for regression.
    The model is pre-tuned with TSDAE."""
    def __init__(self, config, bert1, bert2):
        super(TSDAEDualBERTWithMLP, self).__init__()
        # Load TSDAE-ed BERT models
        self.bert1 = bert1
        self.bert2 = bert2

        # Define MLP head
        hidden_size = config['hidden_size']
        mlp_hidden_size = config['mlp_hidden_size']
        self.mlp = nn.Sequential(
            nn.Linear(2 * hidden_size, mlp_hidden_size),
            nn.ReLU(),
            nn.Linear(mlp_hidden_size, 1)  # Regression output
        )

    def forward(self, input1, attention_mask1, input2, attention_mask2):
        # idea from: https://github.com/UKPLab/sentence-transformers/issues/2494
        # Forward pass through BERT1
        input_dict1 = {
            'input_ids': input1,
            'attention_mask': attention_mask1
        }
        cls1 = self.bert1(input_dict1)['sentence_embedding']

        # Forward pass through BERT2
        input_dict2 = {
            'input_ids': input2,
            'attention_mask': attention_mask2
        }
        cls2 = self.bert2(input_dict2)['sentence_embedding']

        # Concatenate CLS embeddings
        combined_cls = torch.cat([cls1, cls2], dim=-1)  # Shape: [batch_size, 2 * hidden_size]

        # Pass through MLP head
        output = self.mlp(combined_cls)
        return output

#### Training methods

In [ ]:
from torch.utils.data import DataLoader
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


def fit_eval(
    seed,
    model,
    X_train,
    X_test,
    y_train_reg,
    y_test_reg,
    criterion,
    tokenizer,
    config,
    text_col_1,
    text_col_2,
):
    set_seed(seed)

    # Memory cleanup
    memory_cleanup()

    # Unpack config
    learning_rate = config["learning_rate"]
    num_epochs = config["num_epochs"]
    batch_size = config["batch_size"]
    seq_length = config["seq_length"]

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Make datasets
    train_dataset = DualTextDataset(X_train, text_col_1, text_col_2, y_train_reg, tokenizer, seq_length)
    test_dataset = DualTextDataset(X_test, text_col_1, text_col_2, y_test_reg, tokenizer, seq_length)
    # Make dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Training and Evaluation Loop
    history = {
                "train_loss": [],
               "test_loss": [],
                "train_rmse": [],
               "test_rmse": [],
               "train_r2": [],
               "train_r2": [],
               "test_r2": [],
               "train_mae": [],
               "test_mae": [],
               "y_pred": [],
               "y_test": [],
               }

    print('Starting training/eval loop...')
    for epoch in range(num_epochs):
        print('Starting training...')
        # Training Phase
        model.train()
        train_losses = []
        all_preds = []
        all_labels = []
        for batch in train_dataloader:
            inputs1, inputs2, targets = batch
            input1 = inputs1["input_ids"].squeeze(1).to(device)
            attention_mask1 = inputs1["attention_mask"].squeeze(1).to(device)
            input2 = inputs2["input_ids"].squeeze(1).to(device)
            attention_mask2 = inputs2["attention_mask"].squeeze(1).to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(input1, attention_mask1, input2, attention_mask2)
            outputs = outputs.flatten()

            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

            all_preds.extend(outputs.cpu().detach().numpy())
            all_labels.extend(targets.cpu().numpy())

        train_loss = np.mean(train_losses)

        train_r2 = r2_score(all_labels, all_preds)

        train_rmse = mean_squared_error(all_labels, all_preds, squared=False)

        train_mae = mean_absolute_error(all_labels, all_preds)

        history["train_loss"].append(train_loss)
        history["train_r2"].append(train_r2)
        history["train_rmse"].append(train_rmse)
        history["train_mae"].append(train_mae)

        # Evaluation Phase
        print('Epoch done, evaluating...')
        model.eval()
        test_losses = []
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in test_dataloader:
                inputs1, inputs2, targets = batch
                input1 = inputs1["input_ids"].squeeze(1).to(device)
                attention_mask1 = inputs1["attention_mask"].squeeze(1).to(device)
                input2 = inputs2["input_ids"].squeeze(1).to(device)
                attention_mask2 = inputs2["attention_mask"].squeeze(1).to(device)
                targets = targets.to(device)

                outputs = model(input1, attention_mask1, input2, attention_mask2)
                outputs = outputs.flatten()

                loss = criterion(outputs, targets)
                test_losses.append(loss.item())

                all_preds.extend(outputs.cpu().numpy())
                all_labels.extend(targets.cpu().numpy())

        history["y_pred"].append(all_preds)
        history["y_test"].append(all_labels)

        test_loss = np.mean(test_losses)

        test_r2 = r2_score(all_labels, all_preds)

        test_rmse = mean_squared_error(all_labels, all_preds, squared=False)

        test_mae = mean_absolute_error(all_labels, all_preds)

        history["test_loss"].append(test_loss)

        history["test_r2"].append(test_r2)

        history["test_rmse"].append(test_rmse)

        history["test_mae"].append(test_mae)

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, "
              f"Train R2: {train_r2:.4f}, Test R2: {test_r2:.4f}")

        print(f"Epoch {epoch + 1}/{num_epochs}, Train MAE: {train_mae:.4f}, Test MAE: {test_mae:.4f}")

    return model, history

### Training-eval loop with experiments

#### Data preprocessing

##### Define text feature/target columns

In [ ]:
text_col_1 = 'description_no_numbers'
text_col_1_with_prompt = text_col_1 + '_with_prompt' # Add prompt to text column

text_col_2 = 'title_company_location_skills_source' # Merged text column, second feature

target_col = 'log_salary_from' # regression target

##### Create merged title/skills/location/source feature

In [ ]:
df['skills'] = df['skills'].fillna('Не указаны')

title_company_location_skills_feature_template = """
Позиция: {position}
Компания: {company}
Место: {location}
Навыки: {skills}
Источник: {source}
"""

df[text_col_2] = df.apply(lambda x: title_company_location_skills_feature_template.format(
    position=x['title'],
    company=x['company'],
    location=x['location'],
    skills=x['skills'],
    source=x['source']
), axis=1)

##### Add a prompt to the feature 1

In [ ]:
# prompt to be added to feature 1 for [MASK] token embedding regression
# (may be not used here)
prompt = """\
[CLS] Далее указано описание вакансии. \
Судя по описанию, зарплата на этой позиции составляет [MASK].[SEP]\
"""

df[text_col_1_with_prompt] = prompt + df[text_col_1]

#### Training code

In [ ]:
memory_cleanup()

##### Experiment 5: Double BERT, Huber loss: pre-tune with TSDAE

In [ ]:
import torch.nn as nn
import pickle
import warnings
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split


experiment_name = 'double_bert_huber_tsdae'
print(experiment_name.upper())
print('='*100)
print()

# Suppress all FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

seeds = [42, 78687, 123123]
combined_history = {}

# Hyperparameters and configuration
config = {
    "model_name": "sergeyzh/rubert-tiny-turbo",
    "batch_size": 32,
    "seq_length": 1024,
    "hidden_size": 312,
    "mlp_hidden_size": 128,
    "num_epochs": 10,
    "learning_rate": 5e-6,
    "mask_token_index": 17, # position of the [MASK] token in the feature 1 (to be used in MASK token prediction)
}

memory_cleanup()

model_name = config['model_name']
# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset and DataLoader
# Prepare data
X = df[[text_col_1, text_col_1_with_prompt, text_col_2]]
y = df[[target_col,]]

for seed in seeds:
    memory_cleanup()
    print()
    print(f'Starting for seed {str(seed)}...')
    print('-' * 100)
    print()

    combined_history[seed] = {}

    set_seed(seed)

    # Split train-test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    # further split train data into regression train and tsdae train data
    X_train_tsdae, X_tsdae, y_train_tsdae, _ = train_test_split(X_train, y_train, test_size=0.01, random_state=seed)

    # get test and train targets
    y_test_reg = y_test[target_col]
    y_train_tsdae_reg = y_train_tsdae[target_col]

    # NOTE:
    # X_tsdae used for TSDAE
    # X_train_tsdae, y_train_tsdae used for the downstream regression training
    # X_test, y_test used for the final model evaluation
    # y_test_tsdae is not used (hence set to '_')

    # TSDAE part:

    # 1. convert job descriptions into a set of sentences
    # and select 10-60 word sentences as a feature column.
    # 2. Train with TSDAE task on these selected sentences.

    # 1. Create a DataFrame of unique sentences and their lengths for X_tsdae
    unique_sentences = []
    unique_sentence_lengths = []
    for text in X_tsdae[text_col_1]:
        sentences, sentence_lengths = get_sentence_lengths(text)
        unique_sentences.extend(sentences)
        unique_sentence_lengths.extend(sentence_lengths)

    unique_sentences_df = pd.DataFrame({
        'sentence': unique_sentences,
        'length': unique_sentence_lengths
    })

    unique_sentences = unique_sentences_df[(unique_sentences_df.length >= 10) & (unique_sentences_df.length <= 60)]['sentence']

    # get array with features for each bert
    train_sentences_array = [
        unique_sentences.tolist(),
        X_tsdae[text_col_2].tolist(),
    ]

    # 2. Train the models on TSDAE task
    berts_after_tsdae = []
    for index, train_sentences in enumerate(train_sentences_array):
        memory_cleanup()
        berts_after_tsdae.append(train_tsdae_bert(model_name, train_sentences))
    memory_cleanup()

    tsdae_bert1, tsdae_bert2 = berts_after_tsdae

    # Initialize the model for regression
    model = TSDAEDualBERTWithMLP(config, tsdae_bert1, tsdae_bert2)
    model = torch.nn.DataParallel(model).to(device)

    # Loss Function
    criterion = nn.HuberLoss()

    # fit-eval regression
    model, history = fit_eval(
        seed,
        model,
        X_train_tsdae,
        X_test,
        y_train_tsdae_reg,
        y_test_reg,
        criterion,
        tokenizer,
        config,
        text_col_1,
        text_col_2,
    )

    memory_cleanup()

    combined_history[seed] = history

# Display metrics
display_metrics_with_ci(combined_history)

# save the history as pickle
with open(f'./data/history/transfomers_{experiment_name}.pickle', 'wb') as handle:
    pickle.dump(combined_history, handle, protocol=pickle.HIGHEST_PROTOCOL)